In [1]:
from pathlib import Path
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint #GpuUsageLogger
from src.settings import RESULT_DIR
from src.conf.config import CONFIG
from src.dl.lightning_model import SegModel, plot_metrics
from src.dl.model_builder import ModelBuilder
from src.dl.loss_builder import LossBuilder

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/leos/.local/lib/python3.6/site-packages/torchvision/extension.py:11: ResourceWarning:

unclosed file <_io.BufferedReader name='/home/leos/.local/lib/python3.6/site-packages/torchvision/_C.so'>

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package

# Train a pytorch model

- Any model should work if it's wrapped to lightning SegModel wrapper. e.g. smp models

In [2]:
# get the config file
config = CONFIG

# Set the model to do instance or panoptic segmentation (depends what's in the config.py file)
base_model = ModelBuilder.set_model("Unet", config)

# Insert the model to pytorch lightning framework. (Simplifies the training and other stuff)
lightning_model = SegModel.from_conf(base_model, config)

In [3]:
tt_logger = pl_loggers.TestTubeLogger(
    save_dir=RESULT_DIR,
    name=config.experiment_args.model_name,
    version=config.experiment_args.experiment_version
)

checkpoint_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
)

checkpoint_callback = ModelCheckpoint(
    filepath = str(checkpoint_dir),
    save_top_k = 1,
    save_last = True,
    verbose = True, 
    monitor = 'avg_val_loss',
    mode = 'min',
    prefix = ''
)

if config.training_args.resume_training:   
    last_checkpoint_path = lightning_model.fm.model_checkpoint("last")
    trainer = Trainer(
        default_root_dir=config.experiment_args.experiment_root_dir,
        max_epochs=config.training_args.num_epochs, 
        gpus=config.training_args.num_gpus,  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
        #callbacks=[GPUStatsMonitor()],
        resume_from_checkpoint=str(last_checkpoint_path),
        profiler=True,
    )
    
else:
    trainer = Trainer(
        default_root_dir=config.experiment_args.experiment_root_dir,
        max_epochs=config.training_args.num_epochs, 
        gpus=config.training_args.num_gpus,  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
        # callbacks=[GpuUsageLogger()],
        profiler=True,
    )
    

# find the batch size automatically
# new_batch_size = trainer.scale_batch_size(lightning_model)

# Override old batch size
# lightning_model.batch_size = new_batch_size

/home/leos/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:

Checkpoint directory /home/local/leos/Dippa/results/tests/UNET/version_panoptic_hover_nope_test5th exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [4]:
log_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
    / "tf"
)

In [5]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

# Train the model

In [6]:
trainer.fit(lightning_model)

/home/leos/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:

Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given


  | Name      | Type                  | Params
----------------------------------------------------
0 | model     | SmpModelWithClsBranch | 41 M  
1 | criterion | JointPanopticLoss     | 0     
Epoch 0:  95%|█████████▌| 761/799 [04:49<00:14,  2.63it/s, loss=0.343, v_num=t5th]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 799/799 [05:02<00:00,  2.64it/s, loss=0.343, v_num=t5th]
Epoch 00000: avg_val_loss reached 0.35223 (best 0.35223), saving model to /home/local/leos/Dippa/results/tests/UNET/version_panoptic_hover_nope_test5th/epoch=0.ckpt as top 1
Epoch 1:  95%|█████████▌| 761/799 [04:46<00:14,  2.66it/s, loss=0.274, v_num=t5th]
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 38/38 [00:13<00:00,  5.96it/s]
Epoch 00001: avg_val_loss re

In [7]:
# plot_metrics(conf=config, metric='accuracy', scale='linear', save=True)

In [8]:
trainer.test(
    model=lightning_model,
    ckpt_path=lightning_model.fm.model_checkpoint("last").as_posix()
)

/home/leos/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:

You're resuming from a checkpoint that ended mid-epoch. This can cause unreliable results if further training is done, consider using an end of epoch checkpoint. 

Testing:  97%|█████████▋| 37/38 [00:14<00:00,  5.94it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_accuracy': tensor(0.9416, device='cuda:0'),
 'avg_test_iou': tensor(0.6090, device='cuda:0'),
 'avg_test_loss': tensor(0.4734, device='cuda:0'),
 'loss': tensor(0.4734, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 38/38 [00:14<00:00,  2.55it/s]


[{'avg_test_loss': 0.4733676314353943,
  'avg_test_accuracy': 0.9415586590766907,
  'avg_test_iou': 0.6089650988578796,
  'loss': 0.4733676314353943}]